In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
import ConnectionConfig as cc
cc.setupEnvironment()
spark = cc.startLocalCluster("Kafka")
spark.getActiveSession()
wordcount_delta_path = "./spark-warehouse/kafkaWordCount"
#spark.sparkContext.setLogLevel("DEBUG")


## Kafka
In this cell reading from a Kafka topic is initiated.
The enabled code gets the data from a docker container running Kafka.
_The commented code gets the data from a Confluent Cloud Kafka instance. At this point the code with Confluent is not working. With debugging on it seems that the connection is made with the server but the server disconnects imeadiately after the connection is made. The error message is not very helpful._

In [2]:
df = (
     spark.readStream
     .format("kafka")
     .option("kafka.bootstrap.servers", "127.0.0.1:29092")
     .option("subscribe", "demo")
     #.option("startingOffsets", "earliest")
     .load()
 )
# spark.sparkContext.setLogLevel("DEBUG")
# df =  (spark.readStream
#      .format("kafka")
#      .option("kafka.bootstrap.servers", "pkc-6ojv2.us-west4.gcp.confluent.cloud:9092")
#      .option("security.protocol", "SASL_SSL")
#      .option("sasl.jaas.config", "org.apache.kafka.common.security.plain.PlainLoginModule required username='PUT API USERNAME HIER password='PUT API KEY PASSWORD HERE';")
#      .option("sasl.mechanism", "PLAIN")
#      #.option("kafka.group.id", "test")
#      .option("client.dns.lookup","use_all_dns_ips")
#      .option("ssl.endpoint.identification.algorithm", "https")
#      .option("session.timeout.ms", "45000")
#      .option("acks", "all")
#      .option("schema.registry.url", "https://psrc-30dr2.us-central1.gcp.confluent.cloud")
#      .option("basci.auth.credentials.source", "USER_INFO")
#      .option("basic.auth.user.info", "PPZSKZFILMHO5GFA:sn0rEL43vd2Uz8EUDwWNST511BM1UoggZW0m1PeiZg0VuEilOPYIBKFUwIe3jNgd")
#      .option("subscribe", "demo")
#      .option("kafka.session.timeout.ms", "10000")
#      .option("startingOffsets", "earliest")
#         .load())

#df.printSchema()

### Converting the data to a dataframe
json_tuple is an sql function that converts a json string to dataframe columns. The columns are named c0, c1, c2.


In [3]:
from pyspark.sql import functions as F

#df.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints").start("kafkaTest")
inter = df.selectExpr("json_tuple(CAST(value as STRING), 'timestamp', 'key', 'value')")
inter.printSchema()
#stream = inter.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints").start("kafkaTest")

root
 |-- c0: string (nullable = true)
 |-- c1: string (nullable = true)
 |-- c2: string (nullable = true)



A string is converted to a timestamp and column names are changed to eventtime, key and value.

In [5]:
wordCountEvents= inter.selectExpr("cast(c0 as timestamp) as eventtime" , "c1 as key", "INT(c2) as value").withWatermark("eventtime",'10 seconds')
wordCountEvents.printSchema()
stream = wordCountEvents.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints1").start("kafkaTest")


root
 |-- eventtime: timestamp (nullable = true)
 |-- key: string (nullable = true)
 |-- value: integer (nullable = true)



### Wordcount aggregation
The data is aggregated by a 10 second window and the key. The result is written to a delta table.
The results are written to a delatable in append mode. This means that the data is added to the table after the window is closed after the watermark is passed.

In [6]:
groupedEvents =wordCountEvents.groupBy(F.window("eventtime", "10 seconds", "10 seconds"), "key").sum("value").withColumnRenamed("sum(value)", "count")
groupedEvents.printSchema()
stream = groupedEvents.writeStream.format("delta").outputMode("append").option("checkpointLocation",".\checkpoints").option("path", wordcount_delta_path).start()
#stream = groupedEvents.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints2").start()


root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- key: string (nullable = true)
 |-- count: long (nullable = true)



In [12]:
stream.stop()

In [10]:
spark.stop()